# Video Moderation - detecting inappropriate information in stored videos

You can use Amazon Rekognition to detect content that is inappropriate, unwanted, or offensive. You can use Rekognition moderation APIs in social media, broadcast media, advertising, and e-commerce situations to create a safer user experience, provide brand safety assurances to advertisers, and comply with local and global regulations.

Amazon Rekognition Video inappropriate or offensive content detection in stored videos is an asynchronous operation. To start detecting inappropriate or offensive content, call [StartContentModeration](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartContentModeration.html). Amazon Rekognition Video publishes the completion status of the video analysis to an Amazon Simple Notification Service topic. If the video analysis is successful, call [GetContentModeration](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_GetContentModeration.html) to get the analysis results. For more information about starting video analysis and getting the results, see [Calling Amazon Rekognition Video operations](https://docs.aws.amazon.com/rekognition/latest/dg/api-video.html). 

This tutorial will show you how to use Amazon Rekognition Moderation API to moderate stored videos, how to extract text from a video using Amazon Rekognition for Text Moderation, and how to transcribe the audio of the video into text for Text Moderation.

![image-moderation-with-text-arc](../images/video-moderation-with-text-audio.png)

- [Step 1: Setup Notebook](#step1)
- [Step 2: Moderate video using Rekognition video moderation API](#step2)
- [Step 3: Moderate text in video](#step3)
- [Step 4: Moderate audio in video](#step4)
- [Step 5: Clean up](#step5)

# Step 1: Setup Notebook <a id="step1"></a>
Run the below cell to install/update Python dependencies if you run the lab using a local IDE. It is optional if you use a SageMaker Studio Juypter Notebook, which already includes the dependencies in the kernel. 

In [ ]:
# First, let's get the latest installations of our dependencies
%pip install -qU pip
%pip install boto3 -qU
%pip install IPython -qU

In [ ]:
import boto3
import sagemaker as sm
import os
import io
import datetime
from IPython.display import HTML, display
import uuid
import json
import time

# variables
data_bucket = sm.Session().default_bucket()
region = boto3.session.Session().region_name

os.environ["BUCKET"] = data_bucket
os.environ["REGION"] = region
role = sm.get_execution_role()

print(f"SageMaker role is: {role}\nDefault SageMaker Bucket: s3://{data_bucket}")

s3=boto3.client('s3', region_name=region)
rekognition=boto3.client('rekognition', region_name=region)
sagemaker = boto3.client('sagemaker', region_name=region)
comprehend = boto3.client('comprehend', region_name=region)
transcribe=boto3.client('transcribe', region_name=region)

For this lab, we will moderate a demo video containing typical inappropriate scenes, such as alcohol, tobacco, and suggestive. The video also has text and audio containing inappropriate information, such as profanity. You can find the video at `datasets/moderation-video.mp4`.

![video-moderation-data](../images/video-moderation-dataset.png)

Now, let's upload the video to the default S3 bucket for Rekognition to access.

In [ ]:
s3_key = 'content-moderation-im/video-moderation/moderation-video.mp4'
s3.upload_file('../datasets/moderation-video.mp4', data_bucket, s3_key)

# Step 2: Moderate video using Rekognition moderation API  <a id="step2"></a>
Call Rekognition **StartContentModeration** API to detect inappropriate information in the video. Rekognition Video moderation API is an asynchronize API that will start a job managed by Rekognition. We will receive a job_id back when we start the job. Then use iteration logic to heart-beat the **GetContentModeration** API to check job status until the job completes.
The below process should take ~1 minute to complete.

However, it is highly recommended to check out the Notification Channel feature which allows you to receive a SNS notification when the detection job is completed instead of periodically polling the status of the detection. Check [boto3 document](https://docs.aws.amazon.com/cli/latest/reference/rekognition/start-content-moderation.html) for details.

In [ ]:
startModerationLabelDetection = rekognition.start_content_moderation(
    Video={
        'S3Object': {
            'Bucket': data_bucket,
            'Name': s3_key,
        }
    },
    #MinConfidence=50
)

moderationJobId = startModerationLabelDetection['JobId']
display("Job Id: {0}".format(moderationJobId))

getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP'
)

while(getContentModeration['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')

    getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP')

display(getContentModeration['JobStatus'])

The below cell will display the video in the browser with a list of hyperlinks. Click on the links to navigate the video to the specific timestamp.

In [ ]:
s3_video_url = s3.generate_presigned_url('get_object', Params={'Bucket': data_bucket, 'Key': s3_key})
video_tag = f"<video id='cccvid1' controls='controls' width='640' height='360' name='Video' src='{s3_video_url}' autoplay></video>"

label_html = ''''''
for label in getContentModeration["ModerationLabels"]:
    if len(label["ModerationLabel"]["ParentName"]) > 0:
        label_html += f'''<a onclick="document.getElementById('cccvid1').currentTime={round(label['Timestamp']/1000)}">[{label['Timestamp']} ms]: 
                {label['ModerationLabel']['Name']}, confidence: {round(label['ModerationLabel']['Confidence'])}</a><br/>
                '''
display(HTML(video_tag))
display(HTML(label_html))

# Step 3: Moderate text in video <a id="step3"></a>

Similarly to the Video moderation API, text detection with Amazon Rekognition Video is also an asynchronous operation. You start text detection by calling StartTextDetection, which returns a job identifier (JobId). Then we will periodically poll the job status by calling GetTextDetection API. Until the job status becomes SUCCEED.

The below process should take ~1 minute to complete.

In [ ]:
getTextDetection = rekognition.start_text_detection(
    Video={
        'S3Object': {
            'Bucket': data_bucket,
            'Name': s3_key,
        }
    }
)

textJobId = getTextDetection['JobId']
display("Job Id: {0}".format(moderationJobId))

getTextDetection = rekognition.get_text_detection(JobId=textJobId)

while(getTextDetection['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')

    getTextDetection = rekognition.get_text_detection(JobId=textJobId)

display(getTextDetection['JobStatus'])

Now, let's display the video and use the hyperlinks to navigate the timestamps where text was detected.

In [ ]:
s3_video_url = s3.generate_presigned_url('get_object', Params={'Bucket': data_bucket, 'Key': s3_key})
video_tag = f"<video id='cccvid2' controls='controls' width='640' height='360' name='Video' src='{s3_video_url}'></video>"

text_html = ''''''
for txt in getTextDetection["TextDetections"]:
    if txt["TextDetection"]["Type"] == 'LINE':
        text_html += f'''<a onclick="document.getElementById('cccvid2').currentTime={round(txt['Timestamp']/1000)}">[{txt['Timestamp']} ms]: 
                {txt["TextDetection"]["DetectedText"]}, confidence: {round(txt["TextDetection"]["Confidence"],2)}</a><br/>
                '''
display(HTML(video_tag))
display(HTML(text_html))

After extracting text from the video, we can then moderate it using Amazon Comprehend. This lab will not cover the text moderation logic. You can refer to the 04-text-moderation module for more examples.

# Step 4: Moderate audio in video <a id="step4"></a>

We have moderated the video and extracted text from the video using Rekognition. Now, we will transcribe the audio to text using Amazon Transcribe.

Call Transcribe **StartTranscriptionJob** API to transcribe the audio to text. Amazon Transcribe StartTranscriptionJob is an asynchronous API that will start a job managed by Transcribe. We will then call the **GetTranscriptionJob** API to check job status until the job completes.

In [ ]:
job_name = 'video_moderation_job1'

transcribe.start_transcription_job(
        TranscriptionJobName = job_name,
        Media = {
            'MediaFileUri': f's3://{data_bucket}/{s3_key}'
        },
        OutputBucketName = data_bucket,
        OutputKey = 'content-moderation-im/video-moderation/audio_output/',
        MediaFormat = 'mp4',
        LanguageCode = 'en-US'
    )

getTranscription = transcribe.get_transcription_job(TranscriptionJobName = job_name)

while(getTranscription['TranscriptionJob']['TranscriptionJobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')

    getTranscription = rekognition.get_transcription_job(TranscriptionJobName = job_name)

display(getTranscription['TranscriptionJob']['TranscriptionJobStatus'])

The job is completed. Transcribe stored the output data to a JSON file in the S3 path specified in the `OutputKey` parameter. We now open the transcribed output JSON file from S3 and check the transcription accuracy. Notice that some of the words may be inappropriate. 

In [ ]:
filename = 'content-moderation-im/video-moderation/audio_output/'+job_name+'.json'
s3_clientobj = s3.get_object(Bucket=data_bucket, Key=filename)
s3_clientdata = s3_clientobj["Body"].read().decode("utf-8")
original = json.loads(s3_clientdata)
output_transcript = original["results"]["transcripts"]
print(output_transcript)

After transcribing text from the audio of the video, we can then moderate it using Amazon Comprehend. This lab will not cover the text moderation logic. You can refer to labs in the 04-text-moderation module for more examples.

# Step 5: Clean up <a id="step5"></a>

Remove the resources that may incur the cost

In [ ]:
# remove the image from s3 bucket
response = s3.delete_object(Bucket=data_bucket, Key=s3_key)
print(response)

# Conclusion
In this lab, we moderated a video (containing text) using Amazon Rekognition and Amazon Comprehend.